<center>
<img src="../../img/ods_stickers.jpg">
## <center>Открытый курс по машинному обучению
Автор материала: Data Science интерн Ciklum, студент магистерской программы CSDS UCU Виталий Радченко. Материал распространяется на условиях лицензии [Creative Commons CC BY-NC-SA 4.0](https://creativecommons.org/licenses/by-nc-sa/4.0/). Можно использовать в любых целях (редактировать, поправлять и брать за основу), кроме коммерческих, но с обязательным упоминанием автора материала.

# <center> Домашнее задание № 5. Решение
## <center> Логистическая регрессия и случайный лес в задаче кредитного скоринга

**Задание 1** В зале суда есть 5 присяжных, каждый из них по отдельности с вероятностью 70% может правильно определить, виновен подсудимый или нет. С какой вероятностью они все вместе вынесут правильный вердикт, если решение принимается большинством голосов?
- 70.00%
- 83.20%
- 83.70%
- 87.50%

Решение:
поскольку большинство голосов – 3, тогда у нас $m = 3,~N = 5,~p = 0.7$. Подставляем в формулу из статьи $$ \large \mu = \sum_{i=3}^{5}C_5^i0.7^i(1-0.7)^{5-i} $$
После подставления и проделывания всех операций получим ответ 83.70%

In [1]:
# отключим предупреждения Anaconda
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd

## Сделаем функцию, которая будет заменять NaN значения на медиану в каждом столбце таблицы 
def fill_nan(table):
    for col in table.columns:
        table[col] = table[col].fillna(table[col].median())
    return table   

## Считываем данные
data = pd.read_csv('../../data/credit_scoring_sample.csv', sep =';')

independent_columns_names = data.columns.values
independent_columns_names = [x for x in data if x != 'SeriousDlqin2yrs']

table = fill_nan(data)

X = table[independent_columns_names]
y = table['SeriousDlqin2yrs']

# Бутстрэп

**Задание 2.** Сделайте интервальную оценку среднего возраста (age) для клиентов, которые просрочили выплату кредита, с 90% "уверенностью". (используйте пример из статьи. Поставьте np.random.seed(0), как это сделано в статье).

In [2]:
def get_bootstrap_samples(data, n_samples):
    # функция для генерации подвыборок с помощью бутстрэпа
    indices = np.random.randint(0, len(data), (n_samples, len(data)))
    samples = data[indices]
    return samples
def stat_intervals(stat, alpha):
    # функция для интервальной оценки
    boundaries = np.percentile(stat, [100 * alpha / 2., 100 * (1 - alpha / 2.)])
    return boundaries

# сохранение в отдельные numpy массивы данных по просрочке
churn = data[data['SeriousDlqin2yrs'] == 1]['age'].values

# ставим seed для воспроизводимости результатов
np.random.seed(0)

# генерируем выборки с помощью бутстрэра и сразу считаем по каждой из них среднее
churn_mean_scores = [np.mean(sample) 
                       for sample in get_bootstrap_samples(churn, 1000)]

#  выводим интервальную оценку среднего
print("Mean interval",  stat_intervals(churn_mean_scores, 0.1))

Mean interval [ 45.71379414  46.12700479]


# Логистическая регрессия

In [3]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, StratifiedKFold

lr = LogisticRegression(random_state=5, class_weight= 'balanced')
parameters = {'C': (0.0001, 0.001, 0.01, 0.1, 1, 10)}
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=5)

**Задание 3.**
Сделайте GridSearch с метрикой "roc-auc" по параметру C. Какое оптимальное значение параметра C получилось?



In [4]:
grid_search = GridSearchCV(lr, parameters, n_jobs=-1, scoring ='roc_auc', cv=skf)
grid_search = grid_search.fit(X, y)
grid_search.best_estimator_

LogisticRegression(C=0.001, class_weight='balanced', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=5,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)

**Задание 4.** 
Можно ли считать лучшую модель устойчивой? (модель считаем устойчивой, если стандартное отклонение на валидации меньше 0.5%)

In [5]:
grid_search.cv_results_['std_test_score'][1]

0.0035278855414391992

In [6]:
grid_search.best_score_

0.79620647690983104

### Определение влияния признаков

**Задание 5.** Определите самый важный признак. Важность признака определяется абсолютным значением его коэффициента. Также нужно масштабировать все признаки, чтобы можно было корректно сравнивать коэффициенты при них.

In [7]:
from sklearn.preprocessing import StandardScaler
lr = LogisticRegression(C=0.001,random_state=5, class_weight= 'balanced')
scal = StandardScaler()
lr.fit(scal.fit_transform(X), y)

pd.DataFrame({'feat': independent_columns_names,
              'coef': lr.coef_.flatten().tolist()}).sort_values(by='coef', ascending=False)

,coef,feat
1,0.724004,NumberOfTime30-59DaysPastDueNotWorse
3,0.517673,NumberOfTimes90DaysLate
4,0.194732,NumberOfTime60-89DaysPastDueNotWorse
6,0.101326,NumberOfDependents
2,-0.024082,DebtRatio
5,-0.162864,MonthlyIncome
0,-0.416304,age


Самый важный признак – NumberOfTime30-59DaysPastDueNotWorse.

**Задание 6.** Посчитайте долю влияния `DebtRatio` на предсказание (Реализуйте функцию [softmax](https://en.wikipedia.org/wiki/Softmax_function)).

In [8]:
print((np.exp(lr.coef_[0]) / np.sum(np.exp(lr.coef_[0])))[2])

0.114205367199


**Задание 7.** 
Давайте посмотрим, как можно интерпретировать влияние наших признаков. Для этого заново оценим логистическую регрессию в абсолютных величинах. После этого посчитайте, во сколько раз увеличатся шансы, что клиент не выплатит кредит, если увеличить возраст на 20 лет при всех остальных равных значениях признаков (теоретический расчет можно посмотреть [здесь](https://www.unm.edu/~schrader/biostat/bio2/Spr06/lec11.pdf)).

In [10]:
lr = LogisticRegression(C=0.001,random_state=5, class_weight= 'balanced')
lr.fit(X, y)

pd.DataFrame({'feat': independent_columns_names,
              'coef': lr.coef_.flatten().tolist()}).sort_values(by='coef', ascending=False)

,coef,feat
1,0.482349,NumberOfTime30-59DaysPastDueNotWorse
3,0.430314,NumberOfTimes90DaysLate
6,0.115356,NumberOfDependents
4,0.065958,NumberOfTime60-89DaysPastDueNotWorse
2,-0.000011,DebtRatio
5,-0.000011,MonthlyIncome
0,-0.018185,age


In [11]:
np.exp(lr.coef_[0][0]*20)

0.69509577331913275

$\exp^{\beta\delta}$ – во столько раз больше шансы, что клиент не выплатит кредит. Где $\delta$ – на сколько делаем прирост. Например, если увеличить возраст на 20 лет, то шансы, что человек не выплатит кредит, увеличатся в 0.69.

# Случайный лес

In [12]:
from sklearn.ensemble import RandomForestClassifier

# Инициализируем случайный лес с 100 деревьями и сбалансированными классами 
rf = RandomForestClassifier(n_estimators=100, n_jobs=-1, random_state=42, 
                            class_weight='balanced')

## Будем искать лучшие параметры среди следующего набора
parameters = {'max_features': [1, 2, 4], 'min_samples_leaf': [3, 5, 7, 9], 'max_depth': [5,10,15]}

## Делаем опять же стратифицированную k-fold валидацию. Инициализация которой должна у вас продолжать храниться в skf

**Задание 8.** На сколько доля верных ответов лучшей модели случайного леса выше аналогичной метрики для логистической регрессии на валидации?

In [13]:
%%time
rf_grid_search = GridSearchCV(rf, parameters, n_jobs=-1, scoring ='roc_auc', cv=skf,
                             verbose=True)
rf_grid_search = rf_grid_search.fit(X, y)
print(rf_grid_search.best_score_ - grid_search.best_score_) # Ответ 5

Fitting 5 folds for each of 36 candidates, totalling 180 fits


[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   12.9s
[Parallel(n_jobs=-1)]: Done 180 out of 180 | elapsed:  2.0min finished


0.0394249713014
CPU times: user 5.86 s, sys: 320 ms, total: 6.18 s
Wall time: 1min 59s


**Задание 9.** Определите, какой признак имеет самое слабое влияние.

In [15]:
independent_columns_names[np.argmin(rf_grid_search.best_estimator_.feature_importances_)]

'NumberOfDependents'

Весь рейтинг важности признаков

In [18]:
pd.DataFrame({'feat': independent_columns_names,
              'coef': rf_grid_search.best_estimator_.feature_importances_}).sort_values(by='coef', ascending=False)

,coef,feat
1,0.301419,NumberOfTime30-59DaysPastDueNotWorse
3,0.278862,NumberOfTimes90DaysLate
4,0.153586,NumberOfTime60-89DaysPastDueNotWorse
0,0.116950,age
2,0.076190,DebtRatio
5,0.058536,MonthlyIncome
6,0.014457,NumberOfDependents


** Задание 10.** Какое наиболее существенное примущество логистической регрессии перед случайным лесом для нашей бизнес-задачи?

- меньше тратится времени для тренировки модели;
- меньше параметров для перебора;
- интепретируемость признаков;
- линейные свойства алгоритма.

В итоге мы получили, что алгоритм случайно леса лучше сработал для нашей задачи скоринга. Доля верных ответов случайного леса почти на 4% выше. Причинами такого результата стали – небольшое количество признаков и свойства случайного леса как композиции.

Но преимущество логистической регрессии в том, что мы можем проинтерпретировать влияние коэффициентов на результат.

# Бэггинг

In [19]:
from sklearn.ensemble import BaggingClassifier
from sklearn.model_selection import cross_val_score, RandomizedSearchCV

parameters = {'max_features': [2, 3, 4], 'max_samples': [0.5, 0.7, 0.9], 
              "base_estimator__C": [0.0001, 0.001, 0.01, 1, 10, 100]}

**Задание 11.** Следующая задача обучить – бэггинг классификатор. В качестве базовых классификаторов возьмите 100 логистических регрессий и на этот раз используйте не GridSearchCV, а RandomizedSearchCV. Так как перебирать все 54 варианта комбинаций долго, то поставьте максимальное число итераций 20 для RandomizedSearchCV. Также не забудьте передать параметр валидации cv и random_state=1. Какая лучшая доля верных ответов получилась?

In [20]:
bg = BaggingClassifier(LogisticRegression(class_weight='balanced'),
                       n_estimators=100, n_jobs=-1, random_state=42)
r_grid_search = RandomizedSearchCV(bg, parameters, n_jobs=-1, 
                                   scoring ='roc_auc', cv=skf, n_iter=20, random_state=1,
                                  verbose=True)
r_grid_search = r_grid_search.fit(X, y)

Fitting 5 folds for each of 20 candidates, totalling 100 fits


[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:  3.4min finished


In [21]:
r_grid_search.best_score_

0.80746778722957357

**Задача 12.** Дайте интерпретацию лучших параметров для бэггинга. Почему именно такие значения оказались лучшими?

- для бэггинга важно использовать как можно меньше признаков
- бэггинг лучше работает на небольших выборках
- меньше корреляция между одиночными моделями
- чем больше признаков, тем меньше теряется информации

Преимущество случайного леса в том, что деревья в композиции слабо коррелируют между собой и это дает максимальную пользу. Аналогично и для бэггинга с логистической регрессией, чем слабее коррелируют между собой одиночные модели, тем выше точность. Поскольку в логистической регрессии практически нет случайности, то мы должны менять наборы признаков и объектов для достижении максимальной декоррелированости наших моделей.